In [2]:
import pandas as pd
import geopandas as gpd
import requests as r
import numpy as np
import seaborn as s
from geopy.geocoders import Nominatim
import folium
from IPython.display import display

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


import sqlalchemy as sql
import pyodbc
from sqlalchemy import create_engine
import urllib




# UDF Flood data dash

In [4]:
@interact(Risk_Address='')
def zip_pivot(Risk_Address):
    
    geolocator = Nominatim(user_agent="TEST")
    location = geolocator.geocode(Risk_Address, addressdetails =True)
    ZIP=location.raw['address']['postcode']
   
    A= (location.latitude, location.longitude)
    B= (location.raw.get('display_name'))
    
    conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=AMSSQL064PV;DATABASE=UDF_FloodData;Trusted_Connection=yes;')

    sql = "Select * From FEMA_999 as F where F.reportedzipcode=" + str(ZIP)
    data = pd.read_sql(sql,conn)
    
    
    sql2 = "select * from [US-Zips] as a where a.ZIP ="+ str(ZIP)
    data2 = pd.read_sql(sql2,conn)
    
    data=data[['yearofloss','occupancytype','floodzone', 
           'amountpaidonbuildingclaim',
           'amountpaidoncontentsclaim' ,'amountpaidonincreasedcostofcomplianceclaim',
          'reportedzipcode',]]
    
    df_occ =pd.DataFrame({'CODE':[1,2,3,4,5],
                      'OCC':['Single Family','Multi Family','Other Residential','Non Residential','Other']})

    
    data=data.merge(df_occ,how='left',left_on='occupancytype',right_on='CODE')
    data=data.merge(data2,how='left',left_on='reportedzipcode',right_on='ZIP')
    
    latlong = location.raw['boundingbox']
    
    
    
    m = folium.Map(
    location= A,
    zoom_start=12,
    tiles='OpenStreetMap')

    tooltip = str(B)

    folium.Marker(A, tooltip=tooltip).add_to(m)
    
    
    
    return(display(B),display(m),display(data.pivot_table(index=["yearofloss",'OCC','COUNTYNAME','TIER','floodzone','ZIP'], 
                            values="amountpaidonbuildingclaim", aggfunc=np.sum, fill_value=0)),latlong,data)
    
    


interactive(children=(Text(value='', description='Risk_Address'), Output()), _dom_classes=('widget-interact',)…

In [5]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [52]:
# Display the associated webpage in a new window
import IPython
url = 'https://hazards-fema.maps.arcgis.com/apps/webappviewer/index.html?id=8b0adb51996444d4879338b5529aa9cd&extent=-80.13165701330592,25.869273242942185,-80.11088598669404,25.878926568313886'
iframe = '<iframe src=' + url + ' width=950 height=700></iframe>'
IPython.display.HTML(iframe)